In [1]:
import os
IN_KAGGLE = 'KAGGLE_URL_BASE' in os.environ
IN_KAGGLE

False

In [2]:
if IN_KAGGLE:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value = user_secrets.get_secret("gittoken2")
    !git clone https://{secret_value}@github.com/moienr/Palm-Segmentation-Phase2.git

In [3]:
if IN_KAGGLE:
    import time
    import os
    sleep_time = 5
    while not os.path.exists("/kaggle/working/Palm-Segmentation-Phase2"):
        print("didn't find the path, wating {sleep_time} more seconds...")
        time.sleep(sleep_time)
    print("path found...")
    import sys
    sys.path.append("/kaggle/working/Palm-Segmentation-Phase2/dataset")
    sys.path.append("/kaggle/working/Palm-Segmentation-Phase2")

In [4]:
%%capture
if IN_KAGGLE:
    !pip install geemap
    !pip install geedim

In [5]:
import ee
import geemap
import importlib
import ast
import pandas as pd

In [6]:
if IN_KAGGLE:
    # Use the service account credentials to authenticate.
    service_account = 'kaggle@ee-mrangzan.iam.gserviceaccount.com'
    credentials = ee.ServiceAccountCredentials(service_account, '/kaggle/working/CropMapping/ee-mrangzan-e0965f83960a.json')

    # Initialize the Earth Engine client.
    ee.Initialize(credentials)

In [7]:
Map = geemap.Map()

In [8]:
import utils.ee_utils as ee_utils

In [9]:
# Reload the module
importlib.reload(ee_utils)

<module 'utils.ee_utils' from 'd:\\python\\Palm-Segmentation-Phase2\\dataset\\utils\\ee_utils.py'>

In [10]:
date1 = '2017-01-01'
date2 = '2018-12-31'

In [11]:
train_rois =       [[[[-115.31769193146317, 36.22181752652169],
           [-115.31769193146317, 36.11206030324993],
           [-115.1982156131038, 36.11206030324993],
           [-115.1982156131038, 36.22181752652169]]],
         [[[-119.45719968971112, 36.39160158576822],
           [-119.45719968971112, 36.284296790317406],
           [-119.33909666236737, 36.284296790317406],
           [-119.33909666236737, 36.39160158576822]]],
         [[[-97.50193135801848, 37.7307848903575],
           [-97.50193135801848, 37.64873677330059],
           [-97.34812276426848, 37.64873677330059],
           [-97.34812276426848, 37.7307848903575]]],
         [[[-82.75581113589928, 27.897612603979184],
           [-82.75581113589928, 27.784680236830116],
           [-82.63908139957115, 27.784680236830116],
           [-82.63908139957115, 27.897612603979184]]],
         [[[-83.2101092110332, 40.01496257705499],
           [-83.2101092110332, 39.91602446807115],
           [-83.09063289267382, 39.91602446807115],
           [-83.09063289267382, 40.01496257705499]]],
         [[[-85.99463623937322, 39.879916791384225],
           [-85.99463623937322, 39.80241509785699],
           [-85.84151429113103, 39.80241509785699],
           [-85.84151429113103, 39.879916791384225]]],
         [[[-102.70767554373788, 38.30145602778896],
           [-102.70767554373788, 38.199001856009126],
           [-102.59506568045663, 38.199001856009126],
           [-102.59506568045663, 38.30145602778896]]],
         [[[-119.36574537378453, 46.98299749485568],
           [-119.36574537378453, 46.89391978027964],
           [-119.25588209253453, 46.89391978027964],
           [-119.25588209253453, 46.98299749485568]]]]
print(len(train_rois))

8


In [12]:
valid_rois =     [[[[-82.81681948554322, 27.905208645753234],
           [-82.81681948554322, 27.861509904001256],
           [-82.76017123114869, 27.861509904001256],
           [-82.76017123114869, 27.905208645753234]]],
         [[[-83.14451017350122, 39.902822093692805],
           [-83.14451017350122, 39.87674306670296],
           [-83.09163846939965, 39.87674306670296],
           [-83.09163846939965, 39.902822093692805]]],
         [[[-86.01211402979132, 39.79611292138111],
           [-86.01211402979132, 39.7570617919499],
           [-85.92422340479132, 39.7570617919499],
           [-85.92422340479132, 39.79611292138111]]],
         [[[-119.15567629831231, 46.991701505973666],
           [-119.15567629831231, 46.94343825727393],
           [-119.09868472116388, 46.94343825727393],
           [-119.09868472116388, 46.991701505973666]]],
         [[[-97.44518995112891, 37.6323935292761],
           [-97.44518995112891, 37.59513491048194],
           [-97.39128827876563, 37.59513491048194],
           [-97.39128827876563, 37.6323935292761]]],
         [[[-102.5872188045079, 38.21305194647663],
           [-102.5872188045079, 38.16906873370259],
           [-102.52919725909774, 38.16906873370259],
           [-102.52919725909774, 38.21305194647663]]],
         [[[-115.17072321346514, 36.14691080068896],
           [-115.17072321346514, 36.109197698485325],
           [-115.12128473690264, 36.109197698485325],
           [-115.12128473690264, 36.14691080068896]]],
         [[[-119.32417341030806, 36.32560891076892],
           [-119.32417341030806, 36.28798187250634],
           [-119.27301831997603, 36.28798187250634],
           [-119.27301831997603, 36.32560891076892]]]]
print(len(valid_rois))

8


In [13]:

# Define area of interest.
# roi = ee.Geometry.Polygon( 
#          [[[-119.45953763458817, 36.39113976465899],
#           [-119.45953763458817, 36.28162029432795],
#           [-119.3400613162288, 36.28162029432795],
#           [-119.3400613162288, 36.39113976465899]]],
#                           )
roi = ee.Geometry.Polygon(valid_rois[0])


dataset = ee.ImageCollection('USDA/NAIP/DOQQ') \
                  .filter(ee.Filter.date('2017-01-01', '2018-12-31')) \
                  .filter(ee.Filter.stringStartsWith('system:index', 'm_')) # Filter added here

falseColor = dataset.select(['N', 'R', 'G']).mosaic().clip(roi)
trueColor = dataset.select(['R', 'G', 'B']).mosaic().clip(roi)
colorVis = {
  'min': 0,
  'max': 255,
}
# Map.setCenter(-82.65495149661814,28.57377937646652, 20)
Map.addLayer(falseColor, colorVis, 'False Color')
Map.addLayer(trueColor, colorVis, 'True Color')
Map.centerObject(roi, 10)

In [14]:
Map

Map(center=[27.8833606994254, -82.7884953583482], controls=(WidgetControl(options=['position', 'transparent_bg…

In [15]:
# get roi from polygon geemap
# roi = Map.draw_last_feature.geometry()
# print(roi)

In [16]:
print(roi.getInfo())

{'type': 'Polygon', 'coordinates': [[[-82.81681948554322, 27.861509904001256], [-82.76017123114869, 27.861509904001256], [-82.76017123114869, 27.905208645753234], [-82.81681948554322, 27.905208645753234], [-82.81681948554322, 27.861509904001256]]]}


In [17]:
sen1_short_print = lambda s2_collection: ee_utils.ee_property_printer(s2_collection, propertie_name_list=['system:time_start','platform_number',
                                                                                           'system:band_names','instrumentMode',                         
                                                                                            'relativeOrbitNumber_stop','relativeOrbitNumber_start',
                                                                                            'cycleNumber'],
                                                             first_is_t_in_millis=True, df_instead_of_print=False)

In [18]:
import os
def check_folder_exists(folder):
    if not os.path.exists(folder):
        print("Creating folder: " + folder)
        os.makedirs(folder)

In [19]:
def download_rgbn(roi, folder="",
                   file_name="s1_56-48.tif",
                   scale=0.6, bands=['B', 'G', 'R', 'N'],
                   proj='EPSG:3857'):
    if not folder == "":
        check_folder_exists(folder)
    dataset = ee.ImageCollection('USDA/NAIP/DOQQ') \
                    .filter(ee.Filter.date('2017-01-01', '2018-12-31')) \
                    .filter(ee.Filter.stringStartsWith('system:index', 'm_')) # Filter added here
                    
    img = dataset.select(bands).mosaic().clip(roi)

    # EPSG:4326 or EPSG:3857
    geemap.download_ee_image(img, folder + file_name, crs=proj, scale=scale, region=roi)


In [23]:
def name_from_roi(roi):
    center = roi.centroid().getInfo()['coordinates']
    x = str(center[0]).replace('.','-')[0:5]
    y = str(center[1]).replace('.','-')[0:5]
    return x + "_" + y + ".tif"

name_from_roi(roi)

'-82-7_27-88.tif'

In [24]:
download_rgbn(roi,file_name=name_from_roi(roi))

-82-7_27-88.tif: |          | 0.00/386M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


In [27]:
# crop_map = ee.Image('JRC/D5/EUCROPMAP/V1/2018').clip(roi)
# geemap.download_ee_image(crop_map, "crop_map"+".tif",crs='EPSG:3857', scale=10,region = roi)

In [26]:
if IN_KAGGLE:
    DATASET_FOLDER = "../../tmp/rgbn_dataset"
else:
    DATASET_FOLDER = "rgbn_dataset"

In [27]:
train_or_valid = "valid"
if train_or_valid == "train":
    rois = train_rois
else:
    rois = valid_rois

In [28]:
for roi in rois:
    roi = ee.Geometry.Polygon(roi)
    download_rgbn(roi, folder=DATASET_FOLDER + "//" , file_name=name_from_roi(roi))

Creating folder: rgbn_dataset//


-82-7_27-88.tif: |          | 0.00/386M (raw) [  0.0%] in 00:00 (eta:     ?)

-83-1_39-88.tif: |          | 0.00/248M (raw) [  0.0%] in 00:00 (eta:     ?)

KeyboardInterrupt: 

In [53]:
from utils import utils
importlib.reload(utils)

<module 'utils.utils' from 'd:\\python\\Palm-Segmentation-Phase2\\dataset\\utils\\utils.py'>

In [54]:
os.listdir(DATASET_FOLDER)

['-82-7_27-88.tif', '-83-1_39-88.tif']

In [55]:
if IN_KAGGLE:
    !du -sh ../../tmp/rgbn_dataset

In [56]:
# ts_dataset
# |-- crop_map
# |-- s1
#     |-- 1
#     |-- 2
#     |-- 3
#     |-- 4
#     |-- 5
#     |-- 6
# |-- s2
#     |-- 1
#     |-- 2
#     |-- 3
#     |-- 4
#     |-- 5
#     |-- 6

# Loop through the folders in the dataset


# Patching

In [57]:
main_in_folder = "../../tmp/rgbn_dataset" if IN_KAGGLE else "rgbn_dataset//"
main_out_folder = "../../tmp/rgbn_dataset_patched//" if IN_KAGGLE else "rgbn_dataset_patched//"
check_folder_exists(main_out_folder)

In [58]:
# finding path available volumes
# volumes = !ls /kaggle/input


In [59]:
utils.patch_folder(main_in_folder, main_out_folder,mute=True, no_overlap=True)

 -82-7_27-88.tif 
shape after fix: (9174, 10511, 4)
range before norm:  0 146.70833409556076 47.596923564948526 238
NaN Ratio: 0.0 Percent
range after norm and NaN removal:   0 146.70833409556076 47.596923564948526 238 
✅  Final Shape-> (35, 41, 256, 256, 4) 
 -83-1_39-88.tif 
shape after fix: (6308, 9810, 4)
range before norm:  0 138.43553702981893 40.35924037348822 255
NaN Ratio: 0.0 Percent
range after norm and NaN removal:   0 138.43553702981893 40.35924037348822 255 
✅  Final Shape-> (24, 38, 256, 256, 4) 


In [ ]:
if IN_KAGGLE:
    !du -sh ../../tmp/rgbn_dataset_patched//

32G	../../tmp/ts_dataset_patched/


In [ ]:
NAME = "rgbn_dataset_" + train_or_valid
NAME

'crop_map_dataset_spain'

In [ ]:
# zip main_out_folder to kaggle/working/crop_map_dataset.zip
if IN_KAGGLE:
    import shutil
    shutil.make_archive("../working/" + NAME, 'zip', main_out_folder)

In [ ]:
if IN_KAGGLE:
    !du -sh ../working/

14G	../working/


In [ ]:
# delete extra files
if IN_KAGGLE:
    !rm -r ../working/Palm-Segmentation-Phase2